In [1]:
from base_classification import *

In [2]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm
# for sampling
import random # use random.choice? and random.sample for interactions

from sklearn.metrics import make_scorer
from sklearn.datasets import make_classification
from sklearn.metrics import *


In [3]:
X, y = make_classification(2000)

In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [5]:
def eval_pipeline(additional_feats, X, y, verbose=True):
    #print(additional_feats)    
    pipeline = additional_feats[:]
    pipeline.append(('SVC', SVC()))
    model = Pipeline(pipeline[:])

    # split data into 10 folds
    kfold = KFold(n_splits=10, shuffle=False, random_state=42)
    results = cross_val_score(model, X, y, cv=kfold)
    if verbose:
        print("Result: {}".format(results.mean()))
    return results.mean()

In [6]:
# take a random sample
eval_pipeline([], X, y)

Result: 0.8965


0.89649999999999996

In [7]:
#### do some iterations

In [8]:
bmars = BMARS(X)

In [9]:
bmars.construct_pipeline(False)

[('union',
  FeatureUnion(n_jobs=1, transformer_list=[('base model', BaseModel())],
         transformer_weights=None))]

In [10]:
# evaluate...
eval_pipeline(bmars.construct_pipeline(False), X, y)

Result: 0.8965


0.89649999999999996

In [11]:
# one iteration of mh
bk, dk, ck = output_prob_state(0)
u = np.random.uniform()
action = output_action(u, bk, dk, ck)

In [12]:
l = 20.62195407652395

In [13]:
basis = bmars.perform_action(action)

In [14]:
proposed_model = BMARS(**bmars.export())

In [15]:
output = bmars_sample_basis(X, list(basis))
output

{'basis': [3, 13], 'knot': array([-0.67531086,  1.05129393]), 'sign': [-1, 1]}

In [16]:
if action == 'birth':
    output = bmars_sample_basis(X, list(basis), {'signs':[-1, 1]})

In [17]:
# do other things for birth
proposed_model.add_basis(**output)

In [18]:
# evaluate...
eval_pipeline( proposed_model.construct_pipeline(False), X, y)

Result: 0.8935


0.89350000000000007

In [19]:
interaction = 2

In [20]:
output_prob_state(30+1, l)

(0.25777442595655153, 0.4, 0.34222557404344844)

In [21]:
accept_proposal_ratio(X, y, l, interaction, bmars, proposed_model, mode='birth')

488799459.18777287

In [22]:
accept_proposal_ratio(X, y, l, interaction, proposed_model, bmars, mode='death')

2.0458287774329328e-09

In [25]:
acceptance_proba(X, y, l, interaction, bmars, proposed_model, mode='birth')

bayes_ratio: 0.988810877527
prior_ratio: 6.1374863323e-09
proposal_ratio: 488799459.188


1.0

In [26]:
acceptance_proba(X, y, l, interaction, proposed_model, bmars, mode='death')

bayes_ratio: 1.00210174057
prior_ratio: 162933153.063
proposal_ratio: 2.04582877743e-09


0.33403391352255835